### Fit T2E traces for Q7_4p83

In [ ]:
from pathlib import Path

import lmfit

from betata.qubit_measurements.qubit import Qubit, load_qubit
from betata.qubit_measurements.traces import T2ETrace, load_t2e_traces, save_t2e_results
from betata.qubit_measurements.fit_t2e_traces.fit_t2e_traces import fit_t2e_trace, plot_t2e_trace, plot_t2e_vs_time

CWD = Path.cwd()

POPN_CONTRAST_THRESHOLD = 0.35

Specify input folder and output file

In [ ]:
qubit_name = "Q7_4p83"
input_folder = CWD / f"data/qubit_measurements/{qubit_name}/T2E_{qubit_name}"
output_folder = CWD / f"out/qubit_measurements/{qubit_name}/T2E_{qubit_name}"
qubit_file = CWD / f"out/qubit_measurements/{qubit_name}.h5"

Load qubit

In [ ]:
qubit: Qubit = load_qubit(qubit_file)

Load traces

In [ ]:
traces: list[T2ETrace] = load_t2e_traces(input_folder)

Fit traces and save plots

In [ ]:
# we need to manually supply params because the T2E curves for this qubit go from 1 -> 0.5 instead of 0 -> 0.5

fit_results = {}
for trace in traces:
    popn = trace.population
    params = lmfit.Parameters()
    params.add("A", value=popn[0] - popn[-1], min=-1, max=1)
    params.add("B", value=popn[-1], min=-1, max=1)
    params.add("T2E", value=250e-6, min=0)

    fit_result = fit_t2e_trace(trace, save_folder=output_folder, params=params)
    fit_results[trace.id] = fit_result

Inspect and exclude certain traces

In [ ]:
traces_to_inspect = []
for trace in traces:
    if trace.id in traces_to_inspect:
        plot_t2e_trace(trace, show_fit=False)

In [ ]:
bad_fits_to_exclude = []

for trace in traces:
    if trace.id in bad_fits_to_exclude:
        trace.is_excluded = True
    elif abs(trace.A) < POPN_CONTRAST_THRESHOLD:
        trace.is_excluded = True
    else:
        trace.is_excluded = False

included_traces = [tr for tr in traces if not tr.is_excluded]

In [ ]:
t2e_vs_time_fig = plot_t2e_vs_time(included_traces, qubit.name)

In [ ]:
for trace in included_traces:
    if trace.T2E > 320e-6 and abs(trace.A) > POPN_CONTRAST_THRESHOLD:
        plot_t2e_trace(trace)

Save fit results to qubit file

In [ ]:
save_t2e_results(included_traces, qubit)